In [1]:
#!pip install openai
#!pip install langchain-core==0.2.17
#!pip install langchain-openai==0.1.17
#!pip install pydantic==1.10.8
#!pip install pydantic_core==2.18.2
%pip install -U openai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.3/472.3 KB 11.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 28.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 KB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.2/413.2 KB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 19.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 KB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.9 MB/s eta 0:00:0000:010:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 26.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 KB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.6/124.6 KB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 KB 27.8 M

In [2]:
# from langchain_core.pydantic_v1 import BaseModel, Field # 예전 방식

# from pydantic import BaseModel # 새로운 방식

from pydantic.v1 import BaseModel # pydantic.v1 호환이 필요한 경우

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("{topic}에 대한 간단한 농담을 말해줘.")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser # LCEL의 기본 파이프라인

chain.invoke({"topic": "아이스크림"})

/tmp/ipykernel_17145/3759826758.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o-mini")


'왜 아이스크림은 항상 좋은 기분을 유지할까요?\n\n왜냐하면 항상 "스쿱"이 있기 때문이죠! 🍦😄'

In [2]:
%pip install docarray tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.4/128.4 KB 18.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.5
    Not uninstalling urllib3 at /usr/lib/python3/dist-packages, outside environment /usr
    Can't uninstall 'urllib3'. No files were found to uninstall.
Note: you may need to restart the kernel to use updated packages.


In [5]:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["해리슨은 학교에서 일합니다.", "곰은 꿀을 좋아해"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """다음 지문에만 근거해서 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel( # 여러개의 Runnable을 병렬적으로 실행
    {"context": retriever, "question": RunnablePassthrough()} # RunnablePassthrough는 값을 입력받아 그대로 전달하는 객체
)
chain = setup_and_retrieval | prompt | model | output_parser

# retrieval | prompt | model | output. RAG의 기본 파이프라인

/tmp/ipykernel_17145/755495277.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings(),
/home/ubuntu/.local/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [6]:
chain.invoke("해리슨은 어디에서 일하나요?")

'답변: 학교에서 일합니다.'

In [7]:

chain.invoke("곰은 무엇을 좋아하나요?")

'답변: 곰은 꿀을 좋아합니다.'

In [8]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 간단한 농담을 해줘."
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o-mini")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

왜 아이스크림은 항상 좋은 기분을 가지고 있을까요? 

왜냐하면 언제나 "스쿱"이 있기 때문이죠! 🍦😄

In [9]:
chain.batch(["아이스크림", "스파게티", "만두"])

# stream: 입력에 대해 청크를 스트리밍
# invoke: 입력에 대해 체인을 호출
# batch: 입력목록에 대해 체인을 배치로 호출

['아이스크림이 왜 항상 행복할까요? \n\n왜냐하면 항상 "스쿱"이 많으니까요! 🍦😄',
 '왜 스파게티는 학교에서 항상 좋은 성적을 받을까요? \n\n왜냐하면 항상 "알 덴떼" 상태니까요! 🍝😁',
 '만두가 친구에게 말했어요. "너 왜 이렇게 쪼그려 앉아 있어?" 친구가 답했어요. "너무 만두가 먹고 싶어서!"']

In [10]:
await chain.ainvoke("ice cream")

# astream: 비동기로 입력에 대해 청크를 스트리밍
# ainvoke: 비동기로 입력에 대해 체인을 호출
# abatch: 비동기로 입력목록에 대해 체인을 배치로 호출

'왜 아이스크림은 항상 기분이 좋을까요? \n\n왜냐하면 언제나 "스쿱"할 준비가 되어 있으니까요! 🍦😄'

In [11]:
await chain.abatch(
    ["아이스크림", "스파게티", "만두"]
)

['왜 아이스크림은 항상 행복할까요? \n\n왜냐하면 항상 "스쿱"을 받아서요! 🍦😄',
 '왜 스파게티는 항상 파티에 초대받는 걸까요? \n\n왜냐하면, 항상 "소스"가 좋거든요! 🍝😄',
 '왜 만두가 항상 행복할까요? \n\n왜냐하면 속이 꽉 차있어서요! 😄']

In [12]:
chain.input_schema.model_json_schema()

{'properties': {'root': {'title': 'Root'}},
 'required': ['root'],
 'title': 'RunnableParallel<topic>Input',
 'type': 'object'}

In [13]:
chain.output_schema.model_json_schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}